In [81]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Modules

In [82]:
from subprocess import PIPE, run, STDOUT
#from os import environ

#environ['TOMA'] = 'BBBBB'
print(run('module list', stdout=PIPE, stderr=STDOUT, shell=True).stdout.decode('utf-8'))


Currently Loaded Modules:
  1) gcc/6.3.0      3) intel-impi/2017.1.132   5) slurm/17.02.2 (S)
  2) cmake/3.10.0   4) intel-mkl/2018.1.163

  Where:
   S:  Module is Sticky, requires --force to unload or purge




# Parameters

In [84]:
from operator import mul
from functools import reduce
from datetime import datetime

timestamp = datetime.now()

params = {
    # Executable
   'Executable path': '/gpfs/homeb/zam/tnikolov/asynchronator/build/example',
   'Executable name': 'example.laplace3d_jacobi',
    
    # Linear system
   'Matrix': 'laplace3d',
   'Domain partitions': [[0, 50, 100], [0, 50, 100], [0, 50, 100]],
   'Initial guess': 'zero',
   'Exact solution': 'white-noise sine',

    # Solver
   'Iterant': 'jacobi',
   'Steps': 1,
   'Max iterations': 1000000,
   "Tolerance": 1e-4,
   "Timeout [min]": 2    
}

# Device
params['Device'] = 'julia'
params['Processes per node'] = 1
params['Threads per process'] = 256
params['Timestamp'] = timestamp.strftime("%Y-%m-%d %H:%M:%S")
params['Processes grid'] = [len(x) - 1 for x in params['Domain partitions']]
params['Nodes'] = reduce(mul, params['Processes grid'], 1)
params['Processes'] = params['Processes per node'] * params['Nodes']

params

{'Device': 'julia',
 'Domain partitions': [[0, 50, 100], [0, 50, 100], [0, 50, 100]],
 'Exact solution': 'white-noise sine',
 'Executable name': 'example.laplace3d_jacobi',
 'Executable path': '/gpfs/homeb/zam/tnikolov/asynchronator/build/example',
 'Initial guess': 'zero',
 'Iterant': 'jacobi',
 'Matrix': 'laplace3d',
 'Max iterations': 1000000,
 'Nodes': 8,
 'Processes': 8,
 'Processes grid': [2, 2, 2],
 'Processes per node': 1,
 'Steps': 1,
 'Threads per process': 256,
 'Timeout [min]': 2,
 'Timestamp': '2018-04-24 13:15',
 'Tolerance': 0.0001}

# Command

In [86]:
job = ('sbatch '
       '--job-name={executable} '
       '--nodes={nodes} '
       '--ntasks-per-node={procs} '
       '--cpus-per-task={threads} '
       '--time={timeout} '
       '--mail-user=teodor.nikolov22@gmail.com '
       '--mail-type=FAIL '
       '--error=err.txt '
       '--output=out.txt '
       '--wrap=\"srun {execDir}/{executable}\"').format(
    executable=params['Executable name'],
    execDir=params['Executable path'],
    procs=params['Processes per node'],
    nodes=params['Nodes'],
    threads=params['Threads per process'],
    timeout='{:0>2d}:{:0>2d}:00'.format(params['Timeout [min]'] // 60, 
                                        params['Timeout [min]'] % 60)
)
job

'sbatch --job-name=example.laplace3d_jacobi --nodes=8 --ntasks-per-node=1 --cpus-per-task=256 --time=00:02:00 --mail-user=teodor.nikolov22@gmail.com --mail-type=FAIL --error=err.txt --output=out.txt --wrap="srun /gpfs/homeb/zam/tnikolov/asynchronator/build/example/example.laplace3d_jacobi"'

# Submit

In [87]:
from subprocess import run
from pathlib import Path
from json import dump
from datetime import datetime

# Create a run directory
runDir = Path(timestamp.strftime("%Y-%m-%d_%H-%M-%S"))
runDir.mkdir(exist_ok=False)

# Write setup file
with (runDir / 'setup.json').open('w') as setupF: 
    dump(params, setupF)

# Run the job
run(job, cwd=runDir, shell=True)

CompletedProcess(args='sbatch --job-name=example.laplace3d_jacobi --nodes=8 --ntasks-per-node=1 --cpus-per-task=256 --time=00:02:00 --mail-user=teodor.nikolov22@gmail.com --mail-type=FAIL --error=err.txt --output=out.txt --wrap="srun /gpfs/homeb/zam/tnikolov/asynchronator/build/example/example.laplace3d_jacobi"', returncode=0)

# Update records

In [232]:
import pandas as pd
from pathlib import Path
from json import load
from os import stat

# Check for errors
if stat(runDir / 'err.txt').st_size != 0:
    raise ValueError('The error file is not empty! Check err.txt!')
if stat(runDir / 'out.txt').st_size != 0:
    raise ValueError('Something was written to standard output! Check out.txt!')

# Open records
records = pd.read_csv('records.csv')

with (runDir / '0.json').open('r') as recF: 
    data = load(recF)
    del data['CV sample times [s]']
    del data['CV samples']
    data['Info'] = 'N/A' # TODO
    
    records = records.append(data, ignore_index=True)
records.tail()

,Device,Domain partitions,Exact solution,Executable name,Executable path,Initial guess,Iterant,Matrix,Max iterations,Nodes,...,Processes grid,Processes per node,Rank,Relative error,Relative residual,Steps,Threads per process,Timeout [min],Timestamp,Tolerance
0,julia,"[[0, 50, 100], [0, 50, 100], [0, 50, 100]]",white-noise sine,example.laplace3d_jacobi,/gpfs/homeb/zam/tnikolov/asynchronator/build/e...,zero,jacobi,laplace3d,1000000,8,...,"[2, 2, 2]",1,0,0.000337,0.000047,1,256,2,2018-04-24 13:15,0.0001
1,julia,"[[0, 100], [0, 100], [0, 100]]",white-noise sine,example.laplace3d_jacobi,/gpfs/homeb/zam/tnikolov/asynchronator/build/e...,zero,jacobi,laplace3d,1000000,1,...,"[1, 1, 1]",1,0,0.000100,0.000003,1,256,2,2018-04-24 12:08,0.0001


# Save records

In [233]:
records.to_csv('records.csv', index=False)

# Read iterations record

In [235]:
# Read files
itersData = []
for rank in [0]:#range(0, params['Processes']):    
    with (runDir / '{}.json'.format(rank)).open('r') as recF: 
        data = load(recF)
        dataDf = pd.DataFrame(data['CV samples'], 
                              index=data['CV sample times [s]'],
                              columns =['Relative error {}'.format(rank)])
        dataDf.index.name = 'Time [s]'
        itersData.append(dataDf)

itersDf = pd.concat(itersData, axis=1)
itersDf.head()

,Relative error 0
Time [s],
0.000000,1.000000
0.000004,1.000000
0.013112,0.810671
0.026407,0.782430
0.035821,0.771545


# Save iterations record

In [236]:
itersDf.to_csv('{}.csv'.format(runDir.name))

# Clean run directory

In [237]:
for f in runDir.glob('*'):
    f.unlink()
runDir.rmdir()